In [30]:
import numpy as np
import pandas as pd
from datascience import *
import pgeocode
import math

In [56]:
# goldman = Table.read_table('../../just tacos and burritos.csv')
# goldman = goldman.sample(1000)
# goldman.show(15)

In [59]:
# stores = goldman.group('websites')
# stores.show(10)

In [58]:
# cuisines = goldman.group('cuisines')
# cuisines.show(5)

In [5]:
# locs = goldman.group('country')
# locs.show()

country,count
US,1000


In [6]:
goldman = goldman.drop('country').drop('id').drop('dateAdded').drop('dateUpdated').drop('menus.dateSeen').drop('menus.currency')

In [7]:
# goldman.sample(200).show()
categories = []
goldman.column('menus.name').item(9)
for i in range(0, len(goldman.column('menus.name'))):
    name = goldman.column('menus.name').item(i)
    if 'burrito' in name or 'Burrito' in name:
        categories.append('Burrito')
    else:
        categories.append('Taco')

In [8]:
goldman = goldman.with_column('Categories', categories)
for i in range(25, 30):
    name = 'Unnamed: '+str(i)
    goldman.drop(name)

In [60]:
piv = goldman.pivot('cuisines', 'name')
# piv.show(5)

In [11]:
df = goldman.to_df()

In [12]:
df2 = df.drop(['Unnamed: 235'], axis = 1)

In [13]:
for i in range(26, 236):
    name = 'Unnamed: '+str(i)
    df = df.drop([name], axis = 1)

In [14]:
df.to_csv('goldman.csv')

In [15]:
goldman2 = Table.read_table('goldman.csv')
goldman2 = goldman2.drop('Unnamed: 0').drop('menus.category').drop('priceRangeCurrency')

In [16]:
burrito = goldman2.where('Categories', are.equal_to('Burrito'))
taco = goldman2.where('Categories', are.equal_to('Taco'))

In [61]:
# burrito.show(10)
# taco.show(10)

In [62]:
burrito_rat = burrito.group('city')
# burrito_rat.show(5)

In [63]:
taco_rat = taco.group('city')
# taco_rat.show(5)

In [64]:
ratio = taco_rat.join('city', burrito_rat, 'city')
# ratio.show(10)

In [21]:
ratio.relabel('count', 'Tacos')
ratio.relabel('count_2', 'Burritos')

city,Tacos,Burritos
Albuquerque,3,4
Alpharetta,2,1
Anaheim,2,1
Atlanta,2,2
Aurora,1,1
Azusa,2,3
Bakersfield,1,2
Baltimore,3,1
Beeville,1,1
Boise,2,2


In [22]:
ratio = ratio.with_columns('Ratio of Burritos/Tacos', ratio.column('Burritos')/ratio.column('Tacos'))

## Question 1

In [66]:
ratio.show(10)

city,count,count_2
Albuquerque,3,4
Alpharetta,2,1
Anaheim,2,1
Atlanta,2,2
Aurora,1,1
Azusa,2,3
Bakersfield,1,2
Baltimore,3,1
Beeville,1,1
Boise,2,2


## #2

In [65]:
# goldman2.group('city')

In [67]:
# goldman = Table.read_table('../../just tacos and burritos.csv')
# # 

In [68]:
# goldman = goldman.sample(10000)
# id_g = goldman.group('city')
# id_g.show(10)

In [69]:
# locations = []
# count = 1
# for i in id_g.column('id'):
#     city = goldman.where('id', are.equal_to(i)).column('city').item(0)
#     print(city, count)
#     locations.append(city)
#     count += 1

In [31]:
# id_g = id_g.with_column('City', locations)

In [70]:
# id_g

In [123]:
def ratio(goldman2):
    burrito = goldman2.where('Categories', are.equal_to('Burrito'))
    taco = goldman2.where('Categories', are.equal_to('Taco'))
    
    burrito_rat = burrito.group('postalCode').relabel('count', 'burrito')
    taco_rat = taco.group('postalCode').relabel('count', 'taco')
    
    ratio = pd.merge(burrito_rat.to_df(), taco_rat.to_df(), how = "outer")
    
    ratio['Ratio of Burritos/Taco'] = ratio['burrito']/ratio['taco']
    ratio.to_csv('training.csv')
    return ratio

In [129]:
def category(goldman):
    # goldman.sample(200).show()
    categories = []
    goldman.column('menus.name').item(9)
    for i in range(0, len(goldman.column('menus.name'))):
        name = goldman.column('menus.name').item(i)
        if 'burrito' in name or 'Burrito' in name:
            categories.append('Burrito')
        else:
            categories.append('Taco')
    goldman = goldman.with_column('Categories', categories)
    df = goldman.to_df()
    
    for i in range(26, 236):
        name = 'Unnamed: '+str(i)
        df = df.drop([name], axis = 1)
    df.to_csv('part1.csv')
    return df

In [36]:
goldman = Table.read_table('../../just tacos and burritos.csv')

In [130]:
goldman = goldman.sample(10000)
f = category(goldman)

g = Table.read_table('part1.csv')
g

Unnamed: 0,id,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.category,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites,Categories
0,AVwcijlNByjofQCxfYAI,2400 Fletcher Dr,"Fast Food Restaurant,Restaurant",Los Angeles,US,Fast Food Restaurants,2016-03-24T09:38:01Z,2017-07-06T11:14:39Z,us/ca/losangeles/2400fletcherdr/558764851,34.106,-118.257,nan,6.84,6.84,nan,USD,"2016-09-08T10:45:31Z,2016-03-24T09:38:01Z",nan,Chicken Taco (2 Soft Shell),Rick's Drive In & Out,90039,USD,0,25,CA,http://www.ricksdriveinandout.com,Taco
1,AVwc-D4Z_7pvs4fz04pF,455 Park Ave,American Restaurant,Worcester,US,nan,2016-06-20T15:24:18Z,2016-07-06T21:11:09Z,us/ma/worcester/455parkave/168359587,42.2545,-71.8254,nan,12,12,nan,USD,2016-06-20T15:24:18Z,"Chipotle-cinnamon dusted rare tuna, lettuce, tomato, avo ...",Tuna Tacos,Peppercorn's Grille & Bar,1610,USD,25,40,MA,nan,Taco
2,AVwd9B-FByjofQCxt-Fv,2524 W Main,"Tex-Mex Restaurant,american restaurant",Durant,US,nan,2015-09-21T09:32:12Z,2016-03-30T03:58:29Z,us/ok/durant/2524wmain/379971047,33.9997,-96.4091,nan,nan,nan,nan,nan,2016-03-30T03:58:29Z,nan,Smothered Carnitas Burrito,Chili's Grill & Bar,74701,USD,25,40,OK,nan,Burrito
3,AVweUwBMIN2L1WUf37bD,1191 First Ave,Restaurant,New York,US,Mexican,2015-10-19T20:44:43Z,2016-08-22T12:00:11Z,us/ny/newyork/1191firstave/-739575213,40.763,-73.9595,nan,5,5,Botanas,USD,2016-04-24T00:00:00Z,nan,Al Pastor Taco,Maya - New York,10065,USD,0,30,NY,http://www.richardsandoval.com/mayany/index.php,Taco
4,AVwdEU4V_7pvs4fz16_T,298 Irving Ave,"Mexican Restaurant,nonclassifiable establishments",Brooklyn,US,nan,2015-10-20T01:39:21Z,2017-07-06T19:33:50Z,us/ny/brooklyn/298irvingave/1766271113,nan,nan,nan,7,7,nan,USD,2016-04-01T06:40:50Z,nan,"Torta, Taco Al Pastor and Soda",Taqueria El Paisa,11237,USD,0,25,NY,nan,Taco
5,AVwd7QbGkufWRAb5-dBJ,2035 W Washington St,"Fast Food Restaurant,Mexican Restaurant,Mexican Restaura ...",Indianapolis,US,nan,2015-10-23T02:42:16Z,2017-07-06T18:35:41Z,us/in/indianapolis/2035wwashingtonst/-864103396,nan,nan,nan,nan,nan,nan,nan,2016-03-21T00:25:04Z,nan,Crispy Potato Soft Taco,Taco Bell,46222,nan,nan,nan,IN,http://tacobell.com,Taco
6,AVwcx_k2ByjofQCxiH4I,4450 Eastgate Blvd,Mexican Restaurant,Cincinnati,US,nan,2016-10-27T02:02:54Z,2016-10-27T02:02:54Z,us/cherrygrove/cincinnati/4450eastgateblvd/-654801229,nan,nan,nan,10.5,10.5,nan,USD,2016-10-27T02:02:54Z,"A flour tortilla stuffed with steak chunks, onions, chee ...",Burrito De Carne Asada,Rincon Mexicano,45245,USD,25,40,Cherry Grove,http://rinconeastgate.com,Burrito
7,AVwdExGnkufWRAb52CvO,816 Boyd Rd,"Taco Place,Azle Restaurants,Restaurant",Azle,US,Mexican,2016-03-29T05:17:59Z,2016-07-06T20:54:33Z,us/tx/azle/816boydrd/-864077243,32.9076,-97.543,nan,5.16,5.16,nan,USD,"2016-03-29T05:17:59Z,2016-05-24T05:47:37Z",nan,"Enchilada, Taco, Rice, Frijoles, Chips",Taco Casa,76020,nan,nan,nan,TX,http://www.tacocasatexas.com,Taco
8,AVwdExGnkufWRAb52CvO,816 Boyd Rd,"Taco Place,Azle Restaurants,Restaurant",Azle,US,Mexican,2016-03-29T05:17:59Z,2016-07-06T20:54:33Z,us/tx/azle/816boydrd/-864077243,32.9076,-97.543,nan,5.16,5.16,nan,USD,"2016-03-29T05:17:59Z,2016-05-24T05:47:37Z",nan,"Enchilada, Taco, Rice, Frijoles, Chips",Taco Casa,76020,nan,nan,nan,TX,http://www.tacocasatexas.com,Taco
9,AVwcz-6dkufWRAb5zSAf,2401 N Meridian St,"Fast Food Restaurant,Mexican Restaurant,Fast Food Restau ...",Indianapolis,US,nan,2015-10-23T01:25:04Z,2017-07-03T11:13:25Z,us/in/indianapolis/2401nmeridianst/-864103396,39.8014,-86.1565,nan,nan,nan,nan,nan,"2015-10-23T02:32:08Z,2015-10-23T01:25:04Z",nan,Crunchy Taco,Taco Bell,46208,USD,0,25,IN,http://tacobell.com,Taco


In [128]:
goldman2 = Table.read_table('goldman.csv')
goldman2 = goldman2.drop('Unnamed: 0').drop('menus.category').drop('priceRangeCurrency')
r = ratio(goldman2)

In [ ]:
model_table = model_table.to_df()
model_table = pd.merge(model_table, taco)

In [80]:
model_table.to_csv('training.csv')

In [67]:
def longlat(goldman):
    longs = []
    lats = []
    nomi = pgeocode.Nominatim('fr')
    for i in range(0, len(goldman.column('latitude'))):
        if math.isnan(goldman.column('latitude').item(i)):
            zipcode = goldman.column('postalCode').item(i)
            lon = nomi.query_postal_code(str(zipcode))['longitude']
            lat = nomi.query_postal_code(str(zipcode))['latitude']
            longs.append(lon)
            lats.append(lat)
            print(lon, lat, goldman.column('city').item(i), zipcode)
        else:   
            longs.append(goldman.column('longitude').item(i))
            lats.append(goldman.column('latitude').item(i))
            
    return [longs,lats]

In [167]:
zips = Table.read_table('zipcodes.csv')


In [165]:
lala = Table.read_table('r.csv')
lala = lala.drop('Unnamed: 0')
lala.column('postalCode')

array(['11201', '11559', '12866', '15931', '16830', '18034', '1901',
       '19119', '19963', '20737', '21146-2530', '21202', '21601', '23690',
       '25311', '2601', '27262', '27292', '28078', '28117', '2906',
       '2908', '29209', '29615', '29910', '30004', '30066', '30093',
       '30097', '30189', '32225', '33033', '33301', '33759', '35957',
       '36092', '38059', '38671', '38841', '40076', '40213', '40517',
       '41042', '42633', '42718', '43221', '44720', '45245', '45320',
       '45690', '46131', '48855', '49509', '52246', '56301', '60110',
       '60402', '60622', '60714', '61761', '62002', '62521', '64050',
       '64850', '6510', '66611', '6901', '7090', '72956', '73099',
       '74021', '7457', '74701', '75051', '75241', '76137', '76504',
       '76950', '77092', '77303', '77511', '77541', '7901', '79107',
       '79902', '80027', '80504', '80524', '80530', '81003', '81050',
       '83402', '83501', '84015', '84041', '84107', '84770', '84780',
       '85224', '85251',

In [166]:
zipzz = Table().with_columns('postalCode', zips.column(0),
                            'Longitude', zips.column(1),
                            'Latitude', zips.column(2))
post = [str(i) for i in zipzz.column('postalCode')]
zipzz = zipzz.drop('postalCode').with_column('postalCode', post)
zipzz.column('postalCode')

array(['501', '544', '601', ..., '99928', '99929', '99950'], dtype='<U5')

In [170]:
final = zipzz.join('postalCode', lala, 'postalCode')   

In [171]:
final.to_csv('formap.csv')